In [221]:
file = 'data/bipartite/movielens/train.in'
N = 10

In [222]:
import tools

In [223]:
train_set, nodes, poss_set, neg_set = tools.sample_bipartite(file, N)

# Choose poss set with less components increasing

In [224]:
from graph_tool.all import *
g = Graph(directed=False)
g.add_vertex(max(nodes) + 1)
for edge in train_set:
    u, w = map(int, edge.split())
    g.add_edge(g.vertex(u), g.vertex(w))

In [225]:
train_set_2 = set()
for edge in train_set:
    u, w = map(int, edge.split())
    if g.vertex(u).out_degree() > 1 and g.vertex(w).out_degree() > 1:
        train_set_2.add(edge)

In [194]:
len(train_set_2)

724

In [195]:
poss_set = tools.sample_structural_poss(train_set_2, int(N * len(train_set) / 100.))

# Get coordinates

In [226]:
from graph_tool.all import *
g = Graph(directed=False)
g.add_vertex(max(nodes) + 1)

<generator object <genexpr> at 0x7ff017984b40>

In [227]:
a = 0
for edge in train_set:
    if edge not in poss_set:
        a += 1
        u, w = map(int, edge.split())
        g.add_edge(g.vertex(u), g.vertex(w))
print 1.0 * a / len(train_set)

0.900000899812


In [228]:
is_bi, part = graph_tool.topology.is_bipartite(g, partition=True)

In [229]:
is_bi

True

In [230]:
groups = g.new_vertex_property("int")

In [231]:
for u in g.vertices():
    groups[u] = part[u]

In [237]:
pos = sfdp_layout(g, groups=groups, gamma=-10, mu=10)

In [238]:
%matplotlib inline

#graph_draw(g, pos=pos, vertex_fill_color=groups)

In [239]:
from sklearn.metrics import roc_auc_score

In [240]:
features = tools.TopologicalFeatures(g, pos)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.dist])
roc_auc_score(Y, X)

0.80625516258475949

In [236]:
features = tools.TopologicalFeatures(g, pos)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.preferential_attachment])
roc_auc_score(Y, X)

0.90306976702040165